# Pinecone DB (프리티어 사용)

In [1]:
!pip install pinecone

In [3]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key='pcsk_5jr92H_5GMqqsEQUWCsZw7Z6dkWxvFUJa4trggYWvNGiRC1KwhH68hHmqQ8FUNhvhqxXgi')
index_name= 'reviewtest'

pc.create_index(
    name=index_name,
    dimension=384,
    metric='cosine',
    spec = ServerlessSpec(
        cloud='aws',
        region='us-east-1'
    )
)

{
    "name": "reviewtest",
    "metric": "cosine",
    "host": "reviewtest-2k5d0sk.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [4]:
reviews = [
    {'id': '1',
     'text': '이 제품 정말 좋아요! 성능이 만족스러워요!', 
     'sentiment': 'positive'},

    {'id': '2',
     'text': '별로에요. 배터리가 너무 빨리 닳아요.', 
     'sentiment': 'negative'},    

    {'id': '3',
     'text': '디자인이 세련되고 가벼워서 휴대성이 좋아요.', 
     'sentiment': 'positive'},

    {'id': '4',
     'text': '화면이 너무 어두워서 야외에서는 사용이 어렵네요.', 
     'sentiment': 'negative'},

    {'id': '5',
     'text': '키보드 타건감이 좋아서 장시간 사용해도 불편하지 않아요.', 
     'sentiment': 'positive'},

    {'id': '6',
     'text': '팬 소음이 너무 커서 조용한 곳에서는 사용하기 힘들어요.', 
     'sentiment': 'negative'},

    {'id': '7',
     'text': '배터리 수명이 길어서 출장이나 외근 시에도 걱정 없어요.', 
     'sentiment': 'positive'},

    {'id': '8',
     'text': '화면 해상도가 낮아서 기대했던 선명함이 부족하네요.', 
     'sentiment': 'negative'},

    {'id': '9',
     'text': '부팅 속도가 빨라서 작업할 때 스트레스가 없어요.', 
     'sentiment': 'positive'},

    {'id': '10',
     'text': '충전기가 너무 크고 무거워서 들고 다니기 불편해요.', 
     'sentiment': 'negative'},

    {'id': '11',
     'text': '터치패드 감도가 좋아서 마우스 없이도 작업하기 편리해요.', 
     'sentiment': 'positive'},

    {'id': '12',
     'text': '노트북 발열이 심해서 오래 사용하면 불편해요.', 
     'sentiment': 'negative'},

    {'id': '13',
     'text': '화면 크기가 적당해서 작업할 때 시인성이 좋아요.', 
     'sentiment': 'positive'},

    {'id': '14',
     'text': '스피커 음질이 별로라서 음악 감상하기 아쉬워요.', 
     'sentiment': 'negative'},

    {'id': '15',
     'text': '가벼워서 백팩에 넣고 다니기에 부담이 없어요.', 
     'sentiment': 'positive'},

    {'id': '16',
     'text': 'USB 포트 개수가 적어서 확장성이 아쉬워요.', 
     'sentiment': 'negative'},

    {'id': '17',
     'text': '고해상도 디스플레이 덕분에 영상 시청할 때 몰입감이 좋아요.', 
     'sentiment': 'positive'},

    {'id': '18',
     'text': '충전 속도가 느려서 급할 때 사용하기 불편해요.', 
     'sentiment': 'negative'},

    {'id': '19',
     'text': 'SSD가 기본 탑재되어 있어 부팅과 프로그램 실행 속도가 빨라요.', 
     'sentiment': 'positive'},

    {'id': '20',
     'text': '웹캠 화질이 너무 낮아서 화상회의할 때 불편하네요.', 
     'sentiment': 'negative'}
]

In [6]:
from sentence_transformers import SentenceTransformer

pc = Pinecone(api_key='pcsk_5jr92H_5GMqqsEQUWCsZw7Z6dkWxvFUJa4trggYWvNGiRC1KwhH68hHmqQ8FUNhvhqxXgi')
idx = pc.Index(index_name)

model = SentenceTransformer('all-MiniLM-L6-v2')


In [7]:
for review in reviews:
    review_embed = model.encode(review['text']).tolist()
    idx.upsert([
        (review['id'],review_embed,
         {'text': review['text'], 'sentiment': review['sentiment']})
    ])

In [9]:
query_text = '이 노트북의 성은능 괜찮나요?'
query_embed = model.encode(query_text).tolist()

results =idx.query(
    vector=query_embed,
    top_k=3,
    include_metadata=True
)

In [14]:
for review in results['matches']:
    metadata = review['metadata']
    print(metadata['text'],'감성:', metadata['sentiment'])

디자인이 세련되고 가벼워서 휴대성이 좋아요. 감성: positive
별로에요. 배터리가 너무 빨리 닳아요. 감성: negative
부팅 속도가 빨라서 작업할 때 스트레스가 없어요. 감성: positive


In [16]:
sentiment_counts = {
    'positive': 0,
    'negative': 0
}

for review in results['matches']:
    sentiment = review['metadata']['sentiment']
    sentiment_counts[sentiment] += 1
    
print(f"리뷰 분석 결과: {'positive' if sentiment_counts['positive'] > sentiment_counts['negative'] else 'negative'}")

리뷰 분석 결과: positive
